# 🚀 Inferência e Submissão - Fake News Classification - PS Ligia NLP
**Autor:** Carlos Eduardo Falcão Teixera (ceft)  
**Objetivo:** Este notebook representa a fase final do ciclo de vida do nosso modelo. Iremos carregar o nosso classificador baseado em Transformers (DistilBERT ou DistilRoBERTa), pré-processar cegamente o conjunto de dados de teste (garantindo as regras da competição) e gerar o ficheiro `submission.csv` final, otimizado para o F1-Score (Macro Average).

## 1. Setup e Configuração
Nesta secção, instalamos as dependências necessárias, montamos a infraestrutura do Google Drive e configuramos o hardware (GPU) para acelerar o processamento.

In [1]:
# Instalação das bibliotecas-chave (se não existirem no ambiente Colab)
import os
import re
import zipfile
import torch
import numpy as np
import pandas as pd
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import gdown

# Montagem do Google Drive para acesso aos dados e ao modelo
drive.mount('/content/drive')

# Verificação da infraestrutura de GPU (Fundamental para evitar lentidão na inferência)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Dispositivo de Processamento em uso: {device}")

Mounted at /content/drive
✅ Dispositivo de Processamento em uso: cuda


## 2. Carregamento do Modelo
Em projetos corporativos ou de competição, é essencial termos planos de contingência para aceder aos modelos (caso o Drive seja reorganizado ou estejamos a rodar num ambiente partilhado). Aqui implementamos uma abordagem flexível:

* **Opção 1:** Leitura direta a partir do caminho montado no Google Drive.
* **Opção 2:** Leitura a partir de um zip Online.

In [11]:
# GESTÃO DE CARREGAMENTO DO MODELO

USAR_DOWNLOAD_ZIP = True   # False = Google Drive | True = Download Online
TIPO_MODELO = "distilroberta"  # "distilroberta" ou "distilbert"

if not USAR_DOWNLOAD_ZIP:
    print("🔄 Opção 1 selecionada: A carregar modelo diretamente do Google Drive...")

    if TIPO_MODELO == "distilroberta":
        model_path = "/content/drive/MyDrive/PS_Ligia_NLP/best_distilroberta_model"

    elif TIPO_MODELO == "distilbert":
        model_path = "/content/drive/MyDrive/PS_Ligia_NLP/best_distilbert_model"

    else:
        raise ValueError("❌ Tipo de modelo inválido. Use 'distilroberta' ou 'distilbert'.")


else:
    print("🔄 Opção 2 selecionada: A fazer download e a extrair modelo...")

    if TIPO_MODELO == "distilroberta":
        zip_url = "https://drive.google.com/uc?id=1-pIef8Yzp-Ii-GJI_4CE6Tfw1lpPthRV"

    elif TIPO_MODELO == "distilbert":
        zip_url = "https://drive.google.com/uc?id=1MO-xCcPRghG8Lnxb847zfzAdkz_3Obaa"

    else:
        raise ValueError("❌ Tipo de modelo inválido. Use 'distilroberta' ou 'distilbert'.")

    zip_output = f"best_{TIPO_MODELO}_model.zip"
    extract_folder = f"extracted_{TIPO_MODELO}"

    # Download e extração
    if not os.path.exists(extract_folder):
        gdown.download(zip_url, zip_output, quiet=False)

        with zipfile.ZipFile(zip_output, "r") as zip_ref:
            zip_ref.extractall(extract_folder)

    # Busca automática pelo config.json
    model_path = None
    for root, dirs, files in os.walk(extract_folder):
        if "config.json" in files:
            model_path = root
            break

    if model_path is None:
        raise ValueError("❌ Erro: config.json não encontrado. Verifique a integridade do .zip.")

print(f"✅ Modelo selecionado: {TIPO_MODELO}")
print(f"✅ Diretório final do modelo resolvido: {model_path}")

# Carregamento do Tokenizador e do Modelo
print("⏳ A carregar os pesos na memória...")
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.to(device)
print("✅ Tokenizador e Modelo carregados com sucesso!")

🔄 Opção 2 selecionada: A fazer download e a extrair modelo...


Downloading...
From (original): https://drive.google.com/uc?id=1-pIef8Yzp-Ii-GJI_4CE6Tfw1lpPthRV
From (redirected): https://drive.google.com/uc?id=1-pIef8Yzp-Ii-GJI_4CE6Tfw1lpPthRV&confirm=t&uuid=819856ca-02c5-499a-bfbe-cbcb671904a7
To: /content/best_distilroberta_model.zip
100%|██████████| 247M/247M [00:01<00:00, 226MB/s]


✅ Modelo selecionado: distilroberta
✅ Diretório final do modelo resolvido: extracted_distilroberta/best_distilroberta_model
⏳ A carregar os pesos na memória...


Loading weights:   0%|          | 0/104 [00:00<?, ?it/s]

✅ Tokenizador e Modelo carregados com sucesso!


## 3. Carregamento e Limpeza dos Dados de Teste (Kaggle Safe)

No ecossistema Kaggle, a submissão deve ter **estritamente o mesmo número de linhas** do ficheiro `test.csv`. O sistema cruza os seus índices com o gabarito privado. Se utilizarmos o método `.dropna()` para limpar linhas vazias ou dupliciadas, o tamanho da matriz irá divergir e a plataforma rejeitará a submissão.

In [8]:
USAR_DS_ONLINE = True

if not USAR_DS_ONLINE:
  # Carregamento do arquivo de Teste
  test_csv_path = '/content/drive/MyDrive/PS_Ligia_NLP/test.csv'
  df_test = pd.read_csv(test_csv_path)

else:
  # --- Carregando dataset limpo via compartilhamento Online
  url = 'https://drive.google.com/uc?id=1qtUkukHVSMx9EbMMMzJkPkLWxyx3Z-IJ'
  output = 'test.csv'

  try:
    if not os.path.exists(output):
      gdown.download(url, output, quiet=False)

    df_test = pd.read_csv(output)
    print("\n✅ Dataset carregado com sucesso!")
  except Exception as e:
    print(f"❌ Erro ao baixar/ler o arquivo: {e}")

  # ---


print(f"Dimensões do conjunto de Teste Original: {df_test.shape}")

# 1. Preservação da coluna de ID
test_ids = df_test['id'].copy()

# 2. Tratamento para Kaggle (sem dropar linhas)
df_test['title'] = df_test['title'].fillna("")
df_test['text'] = df_test['text'].fillna("")

# 3. Replicação Exata do Pipeline de Limpeza da EDA
def remove_reuters_leakage(text):
    # Remove a tag da Reuters que vazava informação de classes verdadeiras
    return re.sub(r'^.{0,120}?\\(Reuters\\)\s*-\s*', '', str(text))

df_test['text_clean'] = df_test['text'].apply(remove_reuters_leakage)

# 4. Feature Engineering (Concatenação)
# Criando a exata feature de entrada que o modelo viu no treinamento
df_test['input_text'] = df_test['title'] + " - " + df_test['text_clean']

print("✅ Pré-processamento concluído sem perda de linhas.")
display(df_test[['id', 'input_text']].head(3))


✅ Dataset carregado com sucesso!
Dimensões do conjunto de Teste Original: (5712, 5)
✅ Pré-processamento concluído sem perda de linhas.


,id,input_text
0,5398,Obama’s “CLOCK BOY” Comes Back To Texas…After ...
1,5503,LGBT VOLUNTEERS Aren’t Waiting To Be Thrown Of...
2,23151,Colombia authorizes air raids against dissiden...


## 4. Tokenização e Pipeline de Inferência

### Por que utilizamos o `np.argmax()`?
Em tarefas de classificação binária, o nosso modelo (DistilBERT) não produz nativamente zeros e uns. A última camada emite um array de números reais conhecidos como **logits** (ex: `[-1.4, 3.2]`), que representam a força do sinal preditivo para a Classe 0 e Classe 1, respetivamente.

O edital exige estritamente um CSV com a label dura (`0` ou `1`), e não probabilidades de Softmax contínuas. A função matemática `np.argmax(logits, axis=-1)` percorre cada array de logits gerado pelas previsões e **devolve o índice do maior valor**.
Se o maior sinal estiver no índice 0, a predição final será `0` (Real). Se estiver no índice 1, a predição será `1` (Fake).

In [9]:
# Conversão do Pandas DataFrame para um Dataset nativo da Hugging Face
inference_dataset = Dataset.from_pandas(df_test[['input_text']])

# Função de Tokenização padronizada
def tokenize_for_inference(batch):
    return tokenizer(
        batch['input_text'],
        max_length=512,
        padding='max_length',
        truncation=True
    )

print("⏳ A tokenizar o conjunto de teste...")
tokenized_test = inference_dataset.map(tokenize_for_inference, batched=True)

# Configuração do Trainer exclusivo para Inferência
# Utilizamos o eval_batch_size dimensionado para evitar OOM (Out Of Memory) na GPU
inference_args = TrainingArguments(
    output_dir="./temp_results",
    per_device_eval_batch_size=16,
    dataloader_num_workers=2
)

trainer = Trainer(
    model=model,
    args=inference_args
)

print("🚀 A iniciar as predições na GPU (Isso pode levar alguns minutos)...")
# O método predict devolve um objeto NamedTuple onde .predictions contém os logits
raw_predictions = trainer.predict(tokenized_test)
logits = raw_predictions.predictions

# Extração da classe matemática através de Argmax
final_predictions = np.argmax(logits, axis=-1)

print("✅ Predições extraídas com sucesso!")

⏳ A tokenizar o conjunto de teste...


Map:   0%|          | 0/5712 [00:00<?, ? examples/s]

🚀 A iniciar as predições na GPU (Isso pode levar alguns minutos)...


✅ Predições extraídas com sucesso!


## 5. Formatação e Geração do submission.csv
A última etapa consiste em cumprir os requisitos de formatação técnica exigidos pela competição:
1. Ter apenas as colunas `id` e `target`.
2. Não exportar o `index` padrão do Pandas.
3. Classes perfeitamente mapeadas (0 = Real, 1 = Fake).

In [10]:
# Criação do DataFrame de submissão unindo as chaves (ID) preservadas e as predições geradas
submission_df = pd.DataFrame({
    'id': test_ids,
    'target': final_predictions
})

# Verificação de Sanidade (Sanity Check)
assert len(submission_df) == len(df_test), "❌ ERRO CRÍTICO: O número de linhas gerado não corresponde ao arquivo de teste original."

# Visualização de amostragem
print("--- Amostra do Arquivo Final ---")
display(submission_df.head())
print(submission_df.shape)

# Exportação do CSV limpo
submission_path = '/content/drive/MyDrive/PS_Ligia_NLP/submission.csv'
submission_df.to_csv(submission_path, index=False)

print(f"🏆 Submissão perfeitamente formatada e gravada em: {submission_path}")
print("O ficheiro está pronto para ser enviado para a plataforma Kaggle!")

--- Amostra do Arquivo Final ---


,id,target
0,5398,1
1,5503,1
2,23151,0
3,12669,0
4,27864,0


(5712, 2)
🏆 Submissão perfeitamente formatada e gravada em: /content/drive/MyDrive/PS_Ligia_NLP/submission.csv
O ficheiro está pronto para ser enviado para a plataforma Kaggle!
